In [1]:
# pandas version 0.25.2
# numpy version 1.16.5
# sklearn version 0.21.3
# keras version 2.2.4

SyntaxError: invalid syntax (<ipython-input-1-486d8d36d5a6>, line 1)

In [43]:
import pandas, numpy, sklearn, csv, itertools, os

import matplotlib.pyplot as plt

from sklearn.model_selection import KFold, cross_val_score, GridSearchCV, train_test_split
from sklearn.utils import shuffle
from sklearn import svm
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report,confusion_matrix, auc, roc_curve, average_precision_score
from sklearn.externals import joblib
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold

import keras
from keras.layers import Dense
from keras.callbacks import EarlyStopping, CSVLogger
from keras.utils.np_utils import to_categorical 
from keras.models import model_from_json, Sequential


# Functions

In [29]:
def readfile(filename1,filename2, filename3):
    
    names = ['Refno', 'BW (centile)', 'Status',  'Age', 'Ethnics', 'BPD', 'HC', 'Cerebellum_tr', 'CM', 'EFW', 'Nuchal_fold', 
             'AC', 'Femur', 'Va', 'Vp', 'Hem', 'RI (Right)', 'PI (Right)', 'GA_wk', 'GA_day', 'GA_scan_wk']
        
    dataframe = pandas.read_csv(filename1,skiprows=1, delimiter='\,' , names=names, index_col=False )
    dataframe = dataframe.dropna() 
    array = dataframe.values
    array[:,2] = numpy.round(array[:,2])
    X_train_resampled =array
    Y_train_resampled = numpy.abs(array[:,2])

    # load testing file
    dataframe = pandas.read_csv(filename2,skiprows=1, delimiter='\,' , names=names, index_col=False )
    dataframe = dataframe.dropna() 
    array = dataframe.values
    array[:,2] = numpy.round(array[:,2])
    array[:,2] = numpy.abs(array[:,2])
    X_test = array
    Y_test = array[:, 2]

    # load training before oversampled to get the scaler metrics
    dataframe = pandas.read_csv(filename3,skiprows=1, delimiter='\,' , names=names, index_col=False )
    dataframe = dataframe.dropna() 
    array = dataframe.values
    xdata = array
    scaler = StandardScaler().fit(xdata)

    X_train_resampled_scaled = scaler.transform(X_train_resampled)
    X_test_scaled = scaler.transform(X_test)
    
    return X_train_resampled_scaled, X_test_scaled, Y_train_resampled, Y_test

In [30]:
def CV_RF(X,X_test,Y_train_resampled, Y_test, filetosave):
    param_grid = {'n_estimators': [10,20,30], 'max_depth': [4,5,6,7]}

    clf0 = RandomForestClassifier(oob_score=False, random_state=None) 
    grid_clf0 = GridSearchCV(clf0, param_grid, cv=5,return_train_score=True)
    grid_clf0.fit(X, Y_train_resampled)
    
    towritedata = pandas.DataFrame(grid_clf0.cv_results_)
    towritedata.to_csv(filetosave + '\RF_CVResults1.csv')

    clf = RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=4, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=20, n_jobs=1,
            oob_score=False, verbose=0, warm_start=False)

    best_param = list(grid_clf0.best_params_.items())
    
    clf.set_params(**grid_clf0.best_params_)
    
    clf.fit(X, Y_train_resampled)  

    importance = clf.feature_importances_
        
    n_classes = 2
    y_predict_prob =  clf.predict_proba(X_test)

    # Compute ROC curve and ROC area for each class
    fpr = dict()
    tpr = dict()
    roc_auc = dict()
    fpr, tpr, thresholds = sklearn.metrics.roc_curve(Y_test, y_predict_prob[:,1])

    roc_auc = auc(fpr, tpr)
    
    joblib.dump(clf, filetosave + '\RF1.sav')
    
    return best_param[0][1], best_param[1][1], importance, roc_auc


In [31]:
def CV_SVM(X,X_test,Y_train_resampled, Y_test, filetosave):
    
    SVM_model = svm.SVC()
    param_grid_linear = [{'kernel': ['linear'], 'C': numpy.arange(0.1, 10.0, 0.1)}]

    grid = GridSearchCV(estimator= SVM_model, param_grid = param_grid_linear, scoring=None, n_jobs=1, 
                        iid=True, refit=True, cv=5, verbose=1, pre_dispatch='2*n_jobs', error_score='raise', 
                        return_train_score=True)      #cv=None=3-fold cv

    grid.fit(X, Y_train_resampled)
    
    best_params = list(grid.best_params_.items())
    
    towritedata = pandas.DataFrame(grid.cv_results_)
    towritedata.to_csv(filetosave + '\SVM_CVResults.csv')
    
    SVM_model1 = svm.SVC(C=1.0, kernel='rbf', degree=3, gamma='auto', coef0=0.0, shrinking=True, 
                    probability=True, tol=0.001, cache_size=200, class_weight=None, 
                    verbose=False, max_iter=-1, decision_function_shape='ovr', random_state=None)

    SVM_model1.set_params(**grid.best_params_)
    X = X_train_resampled_scaled[:, sequence]
    SVM_model1.fit(X, Y_train_resampled)
    
    importance = SVM_model1.coef_

    joblib.dump(SVM_model1, filetosave + '\SVM.sav')
    
    n_classes = 2
    y_predict_prob =  SVM_model1.predict_proba(X_test)

    # Compute ROC curve and ROC area for each class
    fpr = dict()
    tpr = dict()
    roc_auc = dict()
    fpr, tpr, thresholds = sklearn.metrics.roc_curve(Y_test, y_predict_prob[:,1])

    roc_auc = auc(fpr, tpr)
    
    return best_params[0], importance, roc_auc

In [46]:
def CV_MLP(X,X_test,Y_train_resampled, Y_test, filetosave):
    
    Y_test_MLP = to_categorical(Y_test, num_classes=2)
    Y_train_resampled_MLP = to_categorical(Y_train_resampled, num_classes=2)

    kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=None)
    cvscores0 = []
    cvscores1 = []
    cvscores2 = []
    meancv = []
    
    categorical_accuracy = 0
    k = 0
    epoch = 1

    for train, test in kfold.split(X, Y_train_resampled):
        
        X_train = X[train]
        X_valid = X[test]
        Y_train = Y_train_resampled_MLP[train]
        Y_valid = Y_train_resampled_MLP[test]    
     
        model = MLP_network(X_train, 0)
        csv_logger = CSVLogger(filetosave +  '\lr=0.001_MLP0' + '_' + str(k) + 'fold' + '_metric.csv', append=True, 
                               separator=',')
        model.fit(X_train, Y_train, epochs=epoch, validation_data = [X_valid, Y_valid], shuffle = True, batch_size=64, 
                  verbose=0, callbacks=[csv_logger])
        cvscores0.append(evaluatemodel(model, X_valid, Y_valid))

        model = MLP_network(X_train,1)
        csv_logger = CSVLogger(filetosave +  '\lr=0.001_MLP1' + '_' + str(k) + 'fold' + '_metric.csv', append=True,
                               separator=',')
        model.fit(X_train, Y_train, epochs=epoch, validation_data = [X_valid, Y_valid], shuffle = True, batch_size=64,
                  verbose=0, callbacks=[csv_logger])
        cvscores1.append(evaluatemodel(model, X_valid, Y_valid))
        
        model = MLP_network(X_train,2)
        csv_logger = CSVLogger(filetosave +  '\lr=0.001_MLP2' + '_' + str(k) + 'fold' + '_metric.csv', append=True,
                               separator=',')
        model.fit(X_train, Y_train, epochs=epoch, validation_data = [X_valid, Y_valid], shuffle = True, batch_size=64,
                  verbose=0, callbacks=[csv_logger])
        cvscores2.append(evaluatemodel(model, X_valid, Y_valid))
        
        k += 1
        
    meancv.append(numpy.mean(cvscores0))
    meancv.append(numpy.mean(cvscores1))
    meancv.append(numpy.mean(cvscores2))
    max_index = meancv.index(max(meancv))

    k = 99
    model = MLP_network(X, max_index)
    csv_logger = CSVLogger(filetosave +  '\lr=0.001_MLP' + str(max_index) + '_' + str(k) + 'fold' + '_metric.csv', append=True,
                           separator=',')
    model.fit(X, Y_train_resampled_MLP, epochs=epoch, validation_split = 0.1, shuffle = True, batch_size=64, verbose=0, 
              callbacks=[csv_logger])
    saveMLPmodel(model, filetosave + str('\MLP') + str(max_index) + 'lr=0.001')

    roc_auc = auroc(model, X_test, Y_test_MLP)
        
    return roc_auc 

In [33]:
def auroc(model, X_test, Y_test_MLP):
    
    n_classes = 2
    y_predict_prob =  model.predict_proba(X_test)

    # Compute ROC curve and ROC area for each class
    fpr = dict()
    tpr = dict()
    roc_auc = dict()
    fpr, tpr, thresholds = sklearn.metrics.roc_curve(Y_test_MLP[:,1], y_predict_prob[:,1])
    roc_auc = auc(fpr, tpr)

    return roc_auc

In [34]:
def saveMLPmodel(model, filetosave):
    
    MLPmodel_json = model.to_json()
    with open(filetosave + "lr=0.001.json", "w") as json_file:
        json_file.write(MLPmodel_json)
    # serialize weights to HDF5
    model.save_weights(filetosave + "lr=0.001_weight")
    print("Saved model to disk") 
    

In [35]:
def MLP_network(X_train, network):
    
    input_dim = int(X_train.size/X_train.shape[0])
    
    if network == 0:
        model = Sequential()
        model.add(Dense(input_dim + 2, input_dim=input_dim, activation='relu'))
        model.add(Dense(2, activation='softmax'))
        
    elif network == 1:
        model = Sequential()
        model.add(Dense(input_dim, input_dim=input_dim, activation='relu'))
        model.add(Dense(2, activation='softmax'))
        
    elif network == 2: 
        model = Sequential()
        model.add(Dense(input_dim + 4, input_dim=input_dim, activation='relu'))
        model.add(Dense(input_dim + 2, activation='relu'))
        model.add(Dense(2, activation='softmax'))
    
    keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['categorical_accuracy'])
        
    return model  

In [36]:
def evaluatemodel(model, X_valid, Y_valid):
    
    cvscores = []
    scores = model.evaluate(X_valid, Y_valid, verbose=0)
    cvscores.append(scores[1] * 100)
    
    return numpy.mean(cvscores)
    

In [37]:
def createfolder(filetosave):

    path = "C:\\Users\\sawsn\\Desktop\\Shiernee\\2. Machine Learning Model\\AI_SGA_model\\output" #os.getcwd()  
    print ("The current working directory is %s" % path) 
    path = str(filetosave)
    
    if not os.path.exists(filetosave):    
        try:  
            os.mkdir(path)
            print ("Successfully created the directory %s " % path)       
        except OSError:  
            print ("Creation of the directory %s failed" % path)
    else:  
        print ("%s already exist" % path)            
        
def writetofile(filename, header, datatowrite):
    with filename:
        writer = csv.writer(filename)
        writer.writerow(header)
        for items in datatowrite:
            writer.writerow(items)
    

# Cross Validation on RF and SVM

In [39]:
list_of_features_combinations = [[3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,20],
                                 [16,17,20],
                                 [9,16,17,20],
                                 [9,11,16,17,20],
                                 [9,6,11,12,16,17,20],
                                 [10,16,17,20],
                                 [10,9,16,17,20],
                                 [10,9,11,16,17,20],
                                 [10,9,6,11,12,16,17,20],                               
                                 [10,20],
                                 [3,4,5,6,7,8,9,11,12,13,14,15,16,17,20],
                                 [10,11,17,20],
                                 [9,10, 11, 17, 20]]


roc_auc = numpy.zeros((3,len(list_of_features_combinations)))
feature_importance = numpy.zeros((len(list_of_features_combinations),21))
feature_importance_svm = numpy.zeros((len(list_of_features_combinations),21))
best_param_RF = []

name_ = ['SGA', 'severeSGA']
for i in name_:
    loc = "data\\"
    filename1 = loc + i + '_trainingdata.csv'
    filename2 = loc + i + '_testingdata.csv'
    filename3 = loc + '29Dec2018_trainingdata_beforeoversampled_' + i + '.csv'
    print(filename1)

    [X_train_resampled_scaled, X_test_scaled, Y_train_resampled, Y_test] = readfile(filename1, filename2, filename3)
    
    j=0
    
    for features in list_of_features_combinations:
              
        sequence = features
        print(sequence)
        X = X_train_resampled_scaled[:, sequence]
        X_test = X_test_scaled[:, sequence]
        
        filetosave = 'output\\' + 'output_' + i + '_feature' + str(features)   
        createfolder(filetosave)
        
        [depth, n_estimator, importance, roc_auc[0,j]] = CV_RF(X,X_test,Y_train_resampled, Y_test, filetosave)
        feature_importance[j,sequence] = importance
        [best_param_SVM, importance_svm, roc_auc[1,j]] = CV_SVM(X,X_test,Y_train_resampled, Y_test, filetosave)
        feature_importance_svm[j,sequence] = importance_svm
        best_param_RF.append([depth, n_estimator, best_param_SVM])
        
        j += 1 
        
    myFile = open(i + '_RFimportance1.csv', 'w' ,newline='')             
    header = ['Refno', 'BW (centile)', 'Status',  'Age', 'Ethnics', 'BPD', 'HC', 'Cerebellum_tr', 'CM', 'EFW', 'Nuchal_fold', 
             'AC', 'Femur', 'Va', 'Vp', 'Hem', 'RI (Right)', 'PI (Right)', 'GA_wk', 'GA_day', 'GA_scan_wk']
    writetofile(myFile, header, feature_importance)
    
    
    myFile = open(i + '_SVMimportance1.csv', 'w' ,newline='')             
    header = ['Refno', 'BW (centile)', 'Status',  'Age', 'Ethnics', 'BPD', 'HC', 'Cerebellum_tr', 'CM', 'EFW', 'Nuchal_fold', 
             'AC', 'Femur', 'Va', 'Vp', 'Hem', 'RI (Right)', 'PI (Right)', 'GA_wk', 'GA_day', 'GA_scan_wk']
    writetofile(myFile, header, feature_importance_svm)

    myFile = open(i + '_auroc1.csv', 'w' ,newline='')             
    header = list_of_features_combinations
    writetofile(myFile, header, roc_auc)
    
    myFile = open(i + '_RFparameter1.csv', 'w' ,newline='')             
    header = ['depth', ' no_estimator', 'SVM_C']
    writetofile(myFile, header, best_param_RF)
    
    print('DONE')

data\SGA_trainingdata.csv
[3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 20]
The current working directory is C:\Users\sawsn\Desktop\Shiernee\2. Machine Learning Model\AI_SGA_model\output
Successfully created the directory output\output_SGA_feature[3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 20] 


C:\Users\sawsn\Anaconda3\envs\ML\lib\site-packages\ipykernel_launcher.py:6: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  
C:\Users\sawsn\Anaconda3\envs\ML\lib\site-packages\ipykernel_launcher.py:14: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  
C:\Users\sawsn\Anaconda3\envs\ML\lib\site-packages\ipykernel_launcher.py:23: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
C:\Users\sawsn\Anaconda3\envs\ML\lib\site-packa

Fitting 5 folds for each of 99 candidates, totalling 495 fits


[Parallel(n_jobs=1)]: Done 495 out of 495 | elapsed:   16.4s finished


[16, 17, 20]
The current working directory is C:\Users\sawsn\Desktop\Shiernee\2. Machine Learning Model\AI_SGA_model\output
Successfully created the directory output\output_SGA_feature[16, 17, 20] 


...

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 495 out of 495 | elapsed:    3.0s finished


[9, 10, 11, 17, 20]
The current working directory is C:\Users\sawsn\Desktop\Shiernee\2. Machine Learning Model\AI_SGA_model\output
Successfully created the directory output\output_severeSGA_feature[9, 10, 11, 17, 20] 
Fitting 5 folds for each of 99 candidates, totalling 495 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


DONE


[Parallel(n_jobs=1)]: Done 495 out of 495 | elapsed:    3.0s finished


# CV on MLP Model

In [48]:
list_of_features_combinations = [[3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,20],
                                 [16,17,20],
                                 [9,16,17,20],
                                 [9,11,16,17,20],
                                 [9,6,11,12,16,17,20],
                                 [10,16,17,20],
                                 [10,9,16,17,20],
                                 [10,9,11,16,17,20],
                                 [10,9,6,11,12,16,17,20],                               
                                 [10,20],
                                 [3,4,5,6,7,8,9,11,12,13,14,15,16,17,20],
                                 [10,11,17,20],
                                 [9,10, 11, 17, 20]]

feature_importance = numpy.zeros((len(list_of_features_combinations),21))
best_param_RF = []

name_ = ['SGA', 'severeSGA']
for i in name_:
    loc = "data\\"
    filename1 = loc + i + '_trainingdata.csv'
    filename2 = loc + i + '_testingdata.csv'
    filename3 = loc + '29Dec2018_trainingdata_beforeoversampled_' + i + '.csv'
    print(filename1)
    roc_auc = [] #numpy.zeros((len(list_of_features_combinations)))
    
    [X_train_resampled_scaled, X_test_scaled, Y_train_resampled, Y_test] = readfile(filename1, filename2, filename3)
    
    j=0
    
    for features in list_of_features_combinations:
              
        sequence = features
        print(sequence)
        X = X_train_resampled_scaled[:, sequence]
        X_test = X_test_scaled[:, sequence]
        
        filetosave = 'output\\' + 'output_' + i + '_feature' + str(features)   
        createfolder(filetosave)
       
        roc_auc.append(CV_MLP(X,X_test,Y_train_resampled, Y_test, filetosave))       
        j += 1 
        
    myFile = open(i + '_auroc1.csv', 'a' ,newline='')             
    header = list_of_features_combinations
    with myFile:
        writer = csv.writer(myFile) 
        writer.writerow(roc_auc)    
        
    print('DONE')


data\SGA_trainingdata.csv
[3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 20]
The current working directory is C:\Users\sawsn\Desktop\Shiernee\2. Machine Learning Model\AI_SGA_model\output
output\output_SGA_feature[3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 20] already exist


C:\Users\sawsn\Anaconda3\envs\ML\lib\site-packages\ipykernel_launcher.py:6: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  
C:\Users\sawsn\Anaconda3\envs\ML\lib\site-packages\ipykernel_launcher.py:14: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  
C:\Users\sawsn\Anaconda3\envs\ML\lib\site-packages\ipykernel_launcher.py:23: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.


Saved model to disk
[16, 17, 20]
The current working directory is C:\Users\sawsn\Desktop\Shiernee\2. Machine Learning Model\AI_SGA_model\output
output\output_SGA_feature[16, 17, 20] already exist


KeyboardInterrupt: 

In [ ]:
# Load RF and compute confusion matrix

In [ ]:
from sklearn.metrics import confusion_matrix, roc_curve, auc
import numpy, itertools
import matplotlib.pyplot as plt


class ConfusionMatrix:
    def __init__(self):
        return

    def sensitivity(self, FN, TP):
        """

        :param FN: int
        :param TP: int
        :return: int
        """
        return TP / (TP + FN)

    def specificity(self, TN, FP):
        """

        :param TN: int
        :param FP: int
        :return: int
        """
        return TN / (TN + FP)

    def accuracy(self, TN, FP, FN, TP):
        """

        :param TN: int
        :param FP: int
        :param FN: int
        :param TP: int
        :return: int
        """
        return (TN + TP) / (TN + FP + FN + TP)

    def positive_LR(self, sensitivity, specificity):
        """

        :param sensitivity: float
        :param specificity: float
        :return: float
        """
        return sensitivity / (1 - specificity)

    def negative_LR(self, sensitivity, specificity):
        """

        :param sensitivity: float
        :param specificity: float
        :return: float
        """
        return (1 - sensitivity) / specificity

    def PPV(self, TP, FP):
        """

        :param TP: int
        :param FP: int
        :return: int
        """
        return TP / (TP + FP)

    def NPV(self, TN, FN):
        """

        :param TN: int
        :param FN: int
        :return: int
        """
        return TN / (FN + TN)


In [ ]:
# Load RF and compute statistic metric

In [ ]:
name_ = ['SGA', 'severeSGA']

for i in name_: 
    file_loc = "data\\"
    filename1 = file_loc + i + '_trainingdata.csv'
    filename2 = file_loc + i + '_testingdata.csv'
    filename3 = file_loc + '29Dec2018_trainingdata_beforeoversampled_' + i + '.csv'
    print(filename1)

    fileio = FileIO()
    [X_train_resampled_scaled, X_test_scaled, Y_train_resampled, Y_test] \
        = fileio.readfile(filename1, filename2, filename3)

    stat = ConfusionMatrix()

    writefile = numpy.zeros([numpy.shape(list_of_features_combinations)[0], 12])
    TN, FP, FN, TP = numpy.array([]), numpy.array([]), numpy.array([]), numpy.array([])
    sensitivity = numpy.array([])
    specificity = numpy.array([])
    accuracy = numpy.array([])
    LRP = numpy.array([])
    LRN = numpy.array([])
    PPV = numpy.array([])
    NPV = numpy.array([])

    for j in range(len(list_of_features_combinations)):
        sequence = list_of_features_combinations[j]
        dir_ = "output\\" + 'output_' + i + '_feature' + str(sequence)
        loaded_model_RF = joblib.load(dir_ + '\RF1.sav')

        X = X_test_scaled[:, sequence]
        Y_predict_RF = loaded_model_RF.predict(X)
        TN_tmp, FP_tmp, FN_tmp, TP_tmp = confusion_matrix(Y_test, Y_predict_RF).ravel()

        sensitivity_tmp = stat.sensitivity(FN_tmp, TP_tmp)
        specificity_tmp = stat.specificity(TN_tmp, FP_tmp)
        accuracy_tmp = stat.accuracy(TN_tmp, FP_tmp, FN_tmp, TP_tmp)
        LRP_tmp = stat.positive_LR(sensitivity_tmp, specificity_tmp)
        LRN_tmp = stat.negative_LR(sensitivity_tmp, specificity_tmp)
        PPV_tmp = stat.PPV(TP_tmp, FP_tmp)
        NPV_tmp = stat.NPV(TN_tmp, FN_tmp)

        writefile[j, 0] = TN_tmp
        writefile[j, 1] = FP_tmp
        writefile[j, 2] = FN_tmp
        writefile[j, 3] = TP_tmp
        writefile[j, 4] = sensitivity_tmp
        writefile[j, 5] = specificity_tmp
        writefile[j, 6] = accuracy_tmp
        writefile[j, 7] = LRP_tmp
        writefile[j, 8] = LRN_tmp
        writefile[j, 9] = PPV_tmp
        writefile[j, 10] = NPV_tmp

    myFile = open(i + '_RF_cm1.csv', 'w', newline='')
    header = ['TN', 'FP', 'FN', 'TP', 'sensitivity', 'specificity', 'accuracy', 'Positive LR',
              'Negative LR', 'PPV', 'NPV']
    with myFile:
        writer = csv.writer(myFile)
        writer.writerow(header)
        for items in writefile:
            writer.writerow(items)

    print('RF CM DONE')

In [ ]:
# Load SVM and compute statistic metric

In [ ]:
name_ = ['SGA', 'severeSGA']

for i in name_:  
    file_loc =  "data\\"
    filename1 = file_loc + i + '_trainingdata.csv'
    filename2 = file_loc + i + '_testingdata.csv'
    filename3 = file_loc + '29Dec2018_trainingdata_beforeoversampled_' + i + '.csv'
    print(filename1)

    fileio = FileIO()
    [X_train_resampled_scaled, X_test_scaled, Y_train_resampled, Y_test] \
        = fileio.readfile(filename1, filename2, filename3)

    stat = ConfusionMatrix()

    writefile = numpy.zeros([numpy.shape(list_of_features_combinations)[0], 12])
    TN, FP, FN, TP = numpy.array([]), numpy.array([]), numpy.array([]), numpy.array([])
    sensitivity = numpy.array([])
    specificity = numpy.array([])
    accuracy = numpy.array([])
    LRP = numpy.array([])
    LRN = numpy.array([])
    PPV = numpy.array([])
    NPV = numpy.array([])

    for j in range(len(list_of_features_combinations)):
        sequence = list_of_features_combinations[j]
        dir_ = file_loc = "output\\" + 'output_' + i + '_feature' + str(sequence)

        loaded_model_SVM = joblib.load(dir_ + '\SVM.sav')

        X = X_test_scaled[:, sequence]
        Y_predict_SVM = loaded_model_SVM.predict(X)
        TN_tmp, FP_tmp, FP_tmp, TP_tmp = confusion_matrix(Y_test, Y_predict_SVM).ravel()

        sensitivity_tmp = stat.sensitivity(FP_tmp, TP_tmp)
        specificity_tmp = stat.specificity(TN_tmp, FP_tmp)
        accuracy_tmp = stat.accuracy(TN_tmp, FP_tmp, FN_tmp, TP_tmp)
        LRP_tmp = stat.positive_LR(sensitivity_tmp, specificity_tmp)
        LRN_tmp = stat.negative_LR(sensitivity_tmp, specificity_tmp)
        PPV_tmp = stat.PPV(TP_tmp, FP_tmp)
        NPV_tmp = stat.NPV(TN_tmp, FP_tmp)

        writefile[j, 0] = TN_tmp
        writefile[j, 1] = FP_tmp
        writefile[j, 2] = FN_tmp
        writefile[j, 3] = TP_tmp
        writefile[j, 4] = sensitivity_tmp
        writefile[j, 5] = specificity_tmp
        writefile[j, 6] = accuracy_tmp
        writefile[j, 7] = LRP_tmp
        writefile[j, 8] = LRN_tmp
        writefile[j, 9] = PPV_tmp
        writefile[j, 10] = NPV_tmp

    myFile = open('T' + str(i + 5) + '_SVM_cm.csv', 'w', newline='')
    header = ['TN', 'FP', 'FN', 'TP', 'sensitivity', 'specificity', 'accuracy', 'Positive LR',
              'Negative LR', 'PPV', 'NPV']
    with myFile:
        writer = csv.writer(myFile)
        writer.writerow(header)
        for items in writefile:
            writer.writerow(items)

    print('SVM CM DONE')

# Load MLP and compute statistic metric

In [20]:
model = [['MLP2', 'MLP1', 'MLP0', 'MLP2', 'MLP2', 'MLP0', 'MLP2', 'MLP0', 'MLP0', 'MLP2', 'MLP1', 'MLP0'],            
         ['MLP2', 'MLP0', 'MLP1', 'MLP2', 'MLP1', 'MLP1', 'MLP0', 'MLP2', 'MLP2', 'MLP0', 'MLP2', 'MLP1']]

name_ = ['SGA', 'severeSGA']

for i in range(len(name_)): 
    file_loc =  "data\\"
    filename1 = file_loc + name_[i] + '_trainingdata.csv'
    filename2 = file_loc + name_[i] + '_testingdata.csv'
    filename3 = file_loc + '29Dec2018_trainingdata_beforeoversampled_' + name_[i] + '.csv'
    print(filename1)

    fileio = FileIO()
    [X_train_resampled_scaled, X_test_scaled, Y_train_resampled, Y_test] \
        = fileio.readfile(filename1, filename2, filename3)

    stat = ConfusionMatrix()

    writefile = numpy.zeros([numpy.shape(list_of_features_combinations)[0], 12])
    TN, FP, FN, TP = numpy.array([]), numpy.array([]), numpy.array([]), numpy.array([])
    sensitivity = numpy.array([])
    specificity = numpy.array([])
    accuracy = numpy.array([])
    LRP = numpy.array([])
    LRN = numpy.array([])
    PPV = numpy.array([])
    NPV = numpy.array([])

    for j in range(len(list_of_features_combinations)):
        sequence = list_of_features_combinations[j]
        loc =  "output\\" + 'output_' + i + '_feature' + str(sequence)
        filename = loc + dir_ + '\\' + dir_ + '\\' + str(model[i][j]) + 'lr=0.001lr=0.001.json'
        weight_file = loc + dir_ + '\\' + dir_ + '\\' + str(model[i][j]) + 'lr=0.001lr=0.001_weight'
        json_file = open(filename, 'r')

        loaded_model_json = json_file.read()
        json_file.close()
        loaded_model = model_from_json(loaded_model_json)

        # load weights into new model
        loaded_model.load_weights(weight_file)
        print("Loaded model from disk")

        X = X_test_scaled[:, sequence]
        Y_predict = loaded_model.predict_classes(X, batch_size=10, verbose=1)
        Y_test_MLP = to_categorical(Y_test, num_classes=2)
        TN_tmp, FP_tmp, FN_tmp, TP_tmp = confusion_matrix(Y_test_MLP[:, 1], Y_predict).ravel()

        sensitivity_tmp = stat.sensitivity(FN_tmp, TP_tmp)
        specificity_tmp = stat.specificity(TN_tmp, FP_tmp)
        accuracy_tmp = stat.accuracy(TN_tmp, FP_tmp, FN_tmp, TP_tmp)
        LRP_tmp = stat.positive_LR(sensitivity_tmp, specificity_tmp)
        LRN_tmp = stat.negative_LR(sensitivity_tmp, specificity_tmp)
        PPV_tmp = stat.PPV(TP_tmp, FP_tmp)
        NPV_tmp = stat.NPV(TN_tmp, FN_tmp)

        writefile[j, 0] = TN_tmp
        writefile[j, 1] = FP_tmp
        writefile[j, 2] = FN_tmp
        writefile[j, 3] = TP_tmp
        writefile[j, 4] = sensitivity_tmp
        writefile[j, 5] = specificity_tmp
        writefile[j, 6] = accuracy_tmp
        writefile[j, 7] = LRP_tmp
        writefile[j, 8] = LRN_tmp
        writefile[j, 9] = PPV_tmp
        writefile[j, 10] = NPV_tmp

    myFile = open(name_[i] '_MLP_cm.csv', 'w', newline='')
    header = ['TN', 'FP', 'FN', 'TP', 'sensitivity', 'specificity', 'accuracy', 'Positive LR',
              'Negative LR', 'PPV', 'NPV']
    with myFile:
        writer = csv.writer(myFile)
        writer.writerow(header)
        for items in writefile:
            writer.writerow(items)

    print('MLP CM DONE')


# Logistic regression

In [ ]:
list_of_features_combinations = [[3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,20]]

feature_importance = numpy.zeros((len(list_of_features_combinations), 21))
best_param_RF = []

loc = "data//"
filename1 = loc + 'SGA' + '_trainingdata.csv'
filename2 = loc + 'SGA' + '_testingdata.csv'
filename3 = loc + '29Dec2018_trainingdata_beforeoversampled_' + 'SGA' + '.csv'
print(filename1)

dataframe_training = pandas.read_csv(filename3,skiprows=1, delimiter='\,' , names=names, index_col=False )
dataframe_training = dataframe_training.dropna() 

dataframe_testing = pandas.read_csv(filename2,skiprows=1, delimiter='\,' , names=names, index_col=False )
dataframe_testing = dataframe_testing.dropna() 

names = ['Refno', 'BW (centile)', 'Status',  'Age', 'Ethnics', 'BPD', 'HC', 'Cerebellum_tr', 'CM', 'EFW', 'Nuchal_fold', 
             'AC', 'Femur', 'Va', 'Vp', 'Hem', 'RI (Right)', 'PI (Right)', 'GA_wk', 'GA_day', 'GA_scan_wk']

dataframe_training.columns = names
dataframe_testing.columns = names

dataframe = pd.concat([dataframe_training, dataframe_testing], axis=0, ignore_index=True)

print(dataframe_training.shape)
print(dataframe_testing.shape)
print(dataframe.shape)


In [ ]:
scaler = StandardScaler().fit(dataframe.values)

df = scaler.transform(dataframe.values)
df = pd.DataFrame(df)
df.columns = names
df['Ethnics'] = dataframe['Ethnics']
df['Status'] = dataframe['Status']
df['BW (centile)'] = dataframe['BW (centile)']

In [ ]:
df.head()

In [ ]:
df['NewStatus'] = df['BW (centile)'].apply(lambda x: 'AGA' if x > 10 else ('Severe SGA' if x < 3 else 'SGA'))
df['Status'] = df['NewStatus']
df.drop(['NewStatus'], axis=1, inplace=True)
# df[['BW (centile)', 'NewStatus']]
# plt.plot(df['NewStatus'], df['BW (centile)'], '.')

In [ ]:
import numpy as np
from statsmodels.stats.outliers_influence import variance_inflation_factor

ethnics = pd.get_dummies(df['Ethnics'], drop_first=True)
ethnics.columns = ['Malay', 'Indian', 'Others']

df.drop(['Ethnics'], axis=1, inplace=True)
df = pd.concat([df, ethnics], axis=1)

print(df.shape)


In [ ]:
feature_to_drop=['Refno', 'BW (centile)','GA_wk', 'GA_day']
df.drop(feature_to_drop, axis=1, inplace=True)


In [ ]:
feature_to_drop_for_vif = ['Status', 'EFW']
vif_df = df.drop(feature_to_drop_for_vif,axis=1)
print(vif_df.shape)
print(vif_df.columns)

In [ ]:
vif = pd.DataFrame()
x_to_check = vif_df.values
vif["VIF Factor"] = [variance_inflation_factor(x_to_check, i) for i in range(x_to_check.shape[1])]
vif["features"] = vif_df.columns

In [ ]:
feature_to_drop_for_vif = ['EFW']
X = df.drop(['Status'],axis=1)
Y = df['Status']

X.drop(feature_to_drop_for_vif, axis=1, inplace=True)
X.head()

In [ ]:
import statsmodels.api as sm
mnlogit = sm.MNLogit(Y, X)

# fit the model
result = mnlogit.fit()
print(result.summary())